In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam

df = pd.read_csv('Google_Stock_Price.csv')
df['Close'] = df['Close'].astype(str).str.replace(',', '', regex=False).astype(float)
data = df['Close'].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

seq_len = 60

def create_sequences(dataset, seq_len):
    X, Y = [], []
    for i in range(len(dataset) - seq_len):
        X.append(dataset[i:i+seq_len])
        Y.append(dataset[i+seq_len])
    return np.array(X), np.array(Y)

X_train, y_train = create_sequences(train_data, seq_len)
X_test, y_test = create_sequences(test_data, seq_len)

model = Sequential([
    SimpleRNN(50, activation='tanh', input_shape=(seq_len, 1)),
    Dense(1)
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_data=(X_test, y_test), verbose=0)

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

y_pred = model.predict(X_test, verbose=0)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)

rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
print('Root Mean Squared Error:', rmse)

plt.figure(figsize=(8,4))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted', linestyle='--')
plt.title('Google Stock Price Prediction (RNN)')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.show()